In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import matplotlib.pyplot as plt
from mpl_finance import candlestick2_ohlc
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import pandas as pd
from pandas_datareader import data  
from datetime import datetime
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

In [ ]:
driver = webdriver.Chrome('C:\/chromedriver.exe')
driver.get('https://m.stock.naver.com/index.html#/domestic/capitalization/KOSPI')

In [ ]:
driver.find_element_by_xpath('//*[@id="content"]/div[4]/div[2]/div[2]/div[1]/table/tbody/tr[2]/td[1]/span[1]').click()

In [ ]:
# 주식 크롤링
def stock(n):
    driver = webdriver.Chrome('C:\/chromedriver.exe')
    driver.get('https://m.stock.naver.com/index.html#/domestic/capitalization/KOSPI')
    stock=[]
    for i in range(1,n):

        driver.find_element_by_xpath('//*[@id="content"]/div[4]/div[2]/div[2]/div[1]/table/tbody/tr[{}]/td[1]/span[1]'.format(i)).click()
        time.sleep(1)
        driver.find_element_by_xpath('//*[@id="common_component_tab"]/div/ul/li[4]/a').click()
        for i in range(3):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1)
        time.sleep(1)
        html = driver.page_source
        soup=BeautifulSoup(html, 'html.parser')

        for i in soup.select_one('tbody'):
            company=[]
            company.append(driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/div[1]/span[2]').text)
            company.append(i.select('.VTablePrice_td__PZi0o')[0].text)
            company.append(int(i.select('.VTablePrice_td__PZi0o')[1].text.replace(',','')))
            company.append(int(i.select('.VTablePrice_td__PZi0o')[4].text.replace(',','')))
            company.append(int(i.select('.VTablePrice_td__PZi0o')[5].text.replace(',','')))
            company.append(int(i.select('.VTablePrice_td__PZi0o')[6].text.replace(',','')))
            company.append(int(i.select('.VTablePrice_td__PZi0o')[7].text.replace(',','')))

            
            
            
            
            
            stock.append(company)
    #         print(stock)
        driver.back()

        driver.back()
        time.sleep(1)


    col=['name','date','end','start','high','low','volume']
    df = pd.DataFrame(stock,columns=col)
    df
    df.to_csv("stock.csv", mode='a',header=True,index=False)   

    return df

In [ ]:
stock(11)

In [ ]:
#캔들차트 
def stock_candle(n):
    df=pd.read_csv('./stock.csv')
    df_=df[df['name']==n]
    df_['MA3'] = df_['end'].rolling(3).mean()
    df_['MA5'] = df_['end'].rolling(5).mean()
    df_['MA10'] = df_['end'].rolling(10).mean()
    df_['MA20'] = df_['end'].rolling(20).mean()

    # 그래프 그리기
    fig, ax = plt.subplots(figsize=(10,5))

    ax.set_title('INDEX', fontsize=15)
    ax.set_ylabel("Inter")
    ax.set_xlabel("Date Time")
    ax.plot(df_.index, df_[['end','MA5','MA10']])
    ax.legend(['end','MA5','MA10'])
    plt.show()
    
    
    fig = plt.figure(figsize=(20,10))
    ax = fig.add_subplot(111)
    index = df_.index.astype('str') # 캔들스틱 x축이 str로 들어감

    # 이동평균선 그리기
    ax.plot(index, df_['MA3'], label='MA3', linewidth=0.7)
    ax.plot(index, df_['MA5'], label='MA5', linewidth=0.7)
    ax.plot(index, df_['MA10'], label='MA10', linewidth=0.7)

    # X축 티커 숫자 20개로 제한
    ax.xaxis.set_major_locator(ticker.MaxNLocator(20))

    # 그래프 title과 축 이름 지정
    ax.set_title('INDEX', fontsize=22)
    ax.set_xlabel('Date')

    # 캔들차트 그리기
    candlestick2_ohlc(ax, df_['start'], df_['high'], 
                      df_['low'], df_['end'],
                      width=0.5, colorup='r', colordown='b')
    ax.legend()
    plt.grid()
    plt.show()

In [ ]:
stock_candle('셀트리온')